In [ ]:
!conda install ephem --yes

In [3]:
import ephem
import numpy
import pandas
from SciServer import SciDrive
from astropy import units
from astropy.coordinates import SkyCoord

In [4]:
#SciDrive.directoryList('mpc')
mpc = SciDrive.download('mpc/MPCORB.DAT', localFilePath='mpcorb.dat')
print('ok')

ok


In [25]:
df = pandas.read_fwf('mpcorb.dat')
df = df.set_index('desig')
print('Data read')
df = df.rename(columns = {'epc':'epoch', 'meanAn': 'meanAnomaly', 'perihel':'periheleonArgument'})
df = df.rename(columns = {'nodAL':'nodeAscendingLongitude', 'incl': 'inclination'})
df = df.rename(columns = {'motDail': 'motionDailyMean', 'z':'uncertainty', 'ref': 'reference'})
df = df.rename(columns = {'nOb': 'nObservations', 'nP':'nOppositions', 'arcL':'arcLength'})
df = df.rename(columns = {'rms':'rmsResidual', 'p': 'perturbersCoarse', 'pp':'perturbersPrecise'})
df = df.rename(columns = {'comp':'computerName', 'flg':'flags', 'lastO':'lastObservationDate'})
print('Columns renamed')
#df['lastObservationDate'] = df['lastObservationDate'].apply(lambda x: x[0:4]+'-'+x[4:6]+'-'+x[6:8])
#print('Dates parsed')
#df['lastObservationDate'] = pandas.to_datetime(df['lastObservationDate'], errors='coerce')
#print('Dates dated')
#df['lastObservationYear'] = df['lastObservationDate'].apply(lambda x: x.year)
#print('Years yeared')
#print('ok')

Data read
Columns renamed


In [28]:
df['lastObservationDate'] = df['lastObservationDate'].astype(str)
df['lastObservationDate'] = df['lastObservationDate'].apply(lambda x: x[0:-2])
df['lastObservationDate'] = pandas.to_datetime(df['lastObservationDate'], format='%Y%m%d', errors='coerce')
df['lastObservationYear'] = df['lastObservationDate'].apply(lambda x: x.year)

print('Dates dated')
df['orbit_type'] = 'e'
print('Orbit type set to e')

xephem_column_order = ['name', 'orbit_type', 'inclination']
xephem_column_order += ['nodeAscendingLongitude', 'periheleonArgument', 'a']
xephem_column_order += ['motionDailyMean', 'e', 'meanAnomaly']
xephem_column_order += ['epoch', 'lastObservationYear', 'h', 'g']
xephem_df = df[xephem_column_order]
print('xephem xephemed')


Dates dated
Orbit type set to e
xephem xephemed


In [29]:
# structure of .epb file (what we need to convert to):
# names, {f|e|h|p|E|P}, incli, long-ascending-node, arg-of-perihel, a (mean dist), 
##  mean-daily-motion, eccentricity, mean-anomaly, epoch, equinox-year, mag1, mag2, ang-size

'''
xephem_column_order = ['name', 'orbit_type', 'inclination']
xephem_column_order += ['nodeAscendingLongitude', 'periheleonArgument', 'a']
xephem_column_order += ['motionDailyMean', 'e', 'meanAnomaly']
xephem_column_order += ['epoch', 'lastObservationYear', 'h', 'g']

xephem_df = df[xephem_column_order]
'''

#aline = "C/2002 Y1 (Juels-Holvorcem),e,103.7816,166.2194,128.8232,242.5695,0.0002609,0.99705756,0.0000,04/13.2508/2003,2000,g  6.5,4.0"
#print(aline)
#print('\n')

with open('orbits.csv', 'w') as f:
#    for i in range(0,len(df)):
    for i in range(0,100):
        print('Processing asteroid {:}...'.format(i))
        thisline = ""
        for k in range(0,11):
            thisline += str(xephem_df.iloc[i].values[k]) + ","
        thisline += 'H '+str(xephem_df.iloc[i].values[11]) + ","
        thisline += str(xephem_df.iloc[i].values[12])
        thisline += '\n'
#        print(thisline)
        f.write(thisline)
print('Done')

Processing asteroid 0...
Processing asteroid 1...
Processing asteroid 2...
Processing asteroid 3...
Processing asteroid 4...
Processing asteroid 5...
Processing asteroid 6...
Processing asteroid 7...
Processing asteroid 8...
Processing asteroid 9...
Processing asteroid 10...
Processing asteroid 11...
Processing asteroid 12...
Processing asteroid 13...
Processing asteroid 14...
Processing asteroid 15...
Processing asteroid 16...
Processing asteroid 17...
Processing asteroid 18...
Processing asteroid 19...
Processing asteroid 20...
Processing asteroid 21...
Processing asteroid 22...
Processing asteroid 23...
Processing asteroid 24...
Processing asteroid 25...
Processing asteroid 26...
Processing asteroid 27...
Processing asteroid 28...
Processing asteroid 29...
Processing asteroid 30...
Processing asteroid 31...
Processing asteroid 32...
Processing asteroid 33...
Processing asteroid 34...
Processing asteroid 35...
Processing asteroid 36...
Processing asteroid 37...
Processing asteroid 38

In [34]:
ra_list = []
dec_list = []
for line in open('orbits.csv'):
#    aline = "C/2002 Y1 (Juels-Holvorcem),e,103.7816,166.2194,128.8232,242.5695,0.0002609,0.99705756,0.0000,04/13.2508/2003,2000,g  6.5,4.0"
#    print(line)
#    print(aline)
    yh = ephem.readdb(line)
    yh.compute('2017/9/1')
    ra_list.append(str(yh.ra))
    dec_list.append(str(yh.dec))
#    print(yh.ra)
#    ra_list.append(yh.ra)
#    dec_list.append(yh.dec)
#print(ra_list)
#print(dec_list)
#print(numpy.max(ra_list))
df['ra'] = -9999
df['dec'] = -9999
for i in range(0, len(ra_list)):
    thestr = ra_list[i] + ' ' + dec_list[i]
    a = SkyCoord(thestr, unit='hourangle, deg')
    df[df.iloc[i], 'ra'] = a.ra.deg
    df[df.iloc[i], 'dec'] = a.dec.deg
    
df.head(100)
#print('Done')

TypeError: 'Series' objects are mutable, thus they cannot be hashed

In [ ]:
>>> line = "C/2002 Y1 (Juels-Holvorcem),e,103.7816,166.2194,128.8232,242.5695,0.0002609,0.99705756,0.0000,04/13.2508/2003,2000,g  6.5,4.0"
>>> yh = ephem.readdb(line)
>>> yh.compute('2007/10/1')
>>> print('%.10f' % yh.earth_distance)
14.8046731949
>>> print(yh.mag)
23.96